In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lxml import html
import requests as req
from locale import *

#nao mude
url_base = "https://uspdigital.usp.br/tycho/curriculoLattesBuscar?codpes="
xpath = "//*[@id=\"layout_conteudo\"]/table[1]"

In [ ]:
raw = pd.read_csv("votos.tsv", delimiter="\t")

In [ ]:
#verificar depublicados
raw.NUSP.value_counts()

In [ ]:
#verificar votos validos
lista_valido = []
for index, voto in raw.iterrows():
    if 1 == 1:
        url = url_base + voto.NUSP
        page = req.get(url)
        tree = html.fromstring(page.content)
        resul = tree.xpath(xpath)
        try:
            resul_el = resul[0][1][1].text
        except IndexError:
            lista_valido.append("nao")
            continue
            
        lista_valido.append("sim")
print("Acabou")

In [ ]:
#verificar se é do if
lista_IF = []
for index, voto in raw.iterrows():
    if voto.Valido == "sim":
        url = url_base + voto.NUSP
        page = req.get(url)
        tree = html.fromstring(page.content)
        resul = tree.xpath(xpath)
        try:
            resul_el = resul[0][1][1].text
        except IndexError:
            lista_IF.append("?")
            continue
            
        condicao = (("Instituto de Física" in resul_el) or
                    ("Interunidades FE/IF/IQ/IB" in resul_el) or
                    ("Interunidades Pró-G" in resul_el))
        
        if condicao:
            lista_IF.append("sim")
        else:
            lista_IF.append("não")
    else:
        lista_IF.append("N/A")
print("Acabou")

In [ ]:
#resultado
df1 = pd.DataFrame(lista_IF, columns=["IF"])
df2 = pd.DataFrame(lista_valido, columns=["ValidoTycho"])
resultado = pd.concat([raw, df1], axis=1)
resultado = pd.concat([resultado, df2], axis=1)

In [ ]:
#salvar
resultado.to_csv("resultado.csv")

In [ ]:
resultado[~(resultado.Valido == resultado.ValidoTycho)]